In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
import wandb
import os
from common import train
from common import create_dataloader
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import wandb
wandb.login()
os.environ['http_proxy'] = "http://192.41.170.23:3128" 
os.environ['https_proxy'] = "http://192.41.170.23:3128" 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nutapol-1997. Use `wandb login --relogin` to force relogin


In [2]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA RTX A6000'

In [3]:
from common import EEG


In [7]:
class gamenet(nn.Module):
    def __init__(self):
        
        super(gamenet,self).__init__()
        
        self.l1 = nn.Sequential(
            #in_channel = 16
            #out_channel or Filter size = 100
            #kernel size = (1,25)
            #stride = 1
            #padding = Same
            #Relu
            nn.Conv2d(1,100,kernel_size=(1,25),stride=1,padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(100)
        )
        self.l2 = nn.Sequential(
            #in_channel = 100
            #out_channel or Filter size = 100
            #kernel size = (16,1)
            #stride = 1
            #padding = Valid
            #Relu
            nn.Conv2d(100,100,kernel_size=(1,16),stride=1,padding='valid'),
            #nn.Conv2d(100,100,kernel_size=(64,1),stride=1,padding='valid'),
            nn.ReLU(),
            nn.BatchNorm2d(100)
        )
        self.l3 = nn.Sequential(
            #in_channel = 100
            #out_channel = 50
            #kernel size = (1,30)
            #stride = 1
            #padding = Same
            #Relu
            nn.Conv2d(100,50,kernel_size=(1,30),stride=1,padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(50)
        )
        self.maxpooling1 = nn.MaxPool2d(kernel_size=(1,7),stride=5)
        self.l4 = nn.Sequential(
            #in_channel = 50
            #out_channel = 50
            #kernel size = (1,30)
            #stride = 1
            #padding = Same
            #Relu
            nn.Conv2d(50,50,kernel_size=(1,30),stride=1,padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(50)
        )
        self.maxpooling2 = nn.MaxPool2d(kernel_size=(1,3),stride=2)
        

        self.flatten = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(3050),
            nn.Dropout(0.15)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(3050,1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.15)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.15)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(512,256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.15)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(512,256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.15)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(256,128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.15)
        )
        self.fc5 = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.15)
        )
        self.fc6 = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.15)
        )
        self.softmax = nn.Sequential(
            nn.Linear(32,2),
            nn.Softmax()
        )
    def forward(self,x):
        
        out = self.l1(x)
        out = self.l2(out)
        out = self.l3(out)
        out = self.maxpooling1(out)
        out = self.l4(out)
        out = self.maxpooling2(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        out = self.fc5(out)
        out = self.fc6(out)
        out = self.softmax(out)
        return out

In [8]:
subjects = list(range(1, 51))
#print(subjects)
#[20,31,47,48,49,54,55,59,62]# 



In [9]:
for subject in subjects:
    print("############## {} ########".format(subject))
    
    path = 'physionet.org/files/eegmmidb/1.0.0'
    base_url = 'https://physionet.org/files/eegmmidb/'
    n_type = "EEG_NET_physionet_MI"
    
    # subjects = [1]
    #runs = [3, 4, 7, 8, 11, 12]
    runs = [4, 8, 12]
    subjects = [subject]#[i for i in range(1,40)]
    #subjects = [1]
    # runs = [6,10,14]

    eeg = EEG(path, base_url, subjects, runs)
    raw=eeg.data_to_raw()
    print("Raw done")
    # apply filter
    
    raw=raw.notch_filter([50])
    freq = (1., 40.)
    raw=eeg.filter(freq=freq)
    raw=raw.pick_channels(['C3', 'C4'])
    eeg.create_epochs()
    
    print(raw.ch_names)
    X, y = eeg.get_X_y()
    X = X[:,np.newaxis,:,:]
    print(X.shape)
    #print(X)
    #X2 = X[:, 8:9, :] 
    #X3= X[:, 12:13, :]
    #X = np.concatenate((X2,X3), axis=1)
    #y=y-1
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

    print('train size',X_train.shape, y_train.shape)
    print('Test size',X_test.shape, y_test.shape)
    batch_size = X_train.shape[2]
    train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
    test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
    num_step =math.ceil(len(train_loader.dataset) / batch_size)
    
    
    
    wand = wandb.init(

          # Set the project where this run will be logged
          project="Motor-Imagery", 
          # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
          name="CNN_3layer_2class_S{0}_{1}".format(subject,n_type), 
          # Track hyperparameters and run metadata
          config={
          "learning_rate": 0.000001,
          "architecture": "EEG_NET",
          "dataset": "S{}".format(subjects[0]),
          "epochs": 100000,
          "weightname":"CNN_3layer_2class_S{0}_{1}".format(subject,n_type),
          "num_step_per_epoch" : num_step, 

          }
        )
    wandb.login()
    config = wand.config
    net = gamenet().cuda(0)#ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
    optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss()


    train_loss,valid_loss,train_accuracy,valid_accuracy =train(
        model = net,
        gpu_num = 0,
        train_loader = train_loader,
        test_loader = test_loader,
        optimizer = optimizer  ,
        criterion = criterion ,
        wand = wand
             )


    wandb.alert(
                title='Finish',
                text=f'Finishing training',
            )

############## 1 ########
Raw done
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1057 samples (6.606 sec)

>>> Apply filter.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


>>> Create Epochs.
Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
45 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 45 events and 641 original time points ...
0 bad epochs dropped
['C3', 'C4']
(45, 1, 2, 641)
train size (31, 1, 2, 641) (31,)
Test size (14, 1, 2, 641) (14,)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/home/nutapolt/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 3050])